This notebook applies morphological smoothing and rule-based reclassification using external layers.

### load packages

In [1]:
%matplotlib inline
import os
import datacube
import warnings
import numpy as np
import geopandas as gpd
import pandas as pd
import xarray as xr
import rioxarray
from rasterio.enums import Resampling
from datacube.utils.cog import write_cog
from deafrica_tools.spatial import xr_rasterize
from skimage.morphology import binary_dilation,disk
from skimage.filters.rank import modal
from odc.algo import xr_reproject
import matplotlib.pyplot as plt


/usr/local/lib/python3.8/dist-packages/geopandas/_compat.py:112: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


### input layers and set parameters

In [ ]:
# file paths and attributes
rwanda_tiles_shp='Data/Rwanda_tiles_epsg32736_smaller.shp' # Lesotho tiles shapefile
# rwanda_tiles_shp='Results/Rwanda_random_sampling_AOIs.geojson'
river_network_shp='Data/hotosm_rwa_waterways_lines.shp' # OSM river network data
road_network_shp='Data/hotosm_rwa_roads_lines_filtered.shp' # OSM road network data
google_building_shp='Data/GoogleBuildingLayer_Rwanda.shp' # google bulding layer
hand_raster='Data/hand_Rwanda.tif' # Hydrologically adjusted elevations, i.e. height above the nearest drainage (hand)
classification2021_raster='Results/Land_cover_prediction_Rwanda_2021_using_2015_sheme2_stratified_samples_balanced_smaller_mosaic.tif' # predicted land cover map of 2021
dict_map={'Forest':1,'Grassland':5,'Shrubland':7,'Perennial Cropland':9,'Annual Cropland':10,
          'Wetland':11,'Water Body':12,'Urban Settlement':13} # dictionary of class values corresponding to each class name
output_crs='epsg:32735' # output crs: WGS84/UTM Zone 35S

## load layers

In [ ]:
# load Rwanda tiles and get bounding box
rwanda_tiles=gpd.read_file(rwanda_tiles_shp).to_crs(output_crs) # get bounding boxes of tiles covering Lesotho
tile_bboxes=rwanda_tiles.bounds

# load land cover maps
landcover2021=rioxarray.open_rasterio(classification2021_raster).astype(np.uint8).squeeze() # import land cover map of 2021
landcover2021=landcover2021.rio.reproject(output_crs) # reproject

# load external layers
# OSM road network
road_network=gpd.read_file(road_network_shp).to_crs(output_crs) # import OSM road network data and reproject
road_network=road_network.loc[road_network['surface'].isin(['asphalt', 'paved', 'compacted', 'cobblestone', 
                                                             'concrete', 'metal', 'paving_stones', 
                                                             'paving_stones:30'])] # select road network by attributes
road_network.geometry=road_network.geometry.buffer(10) # buffer the road network by 10m

# OSM river network
river_network=gpd.read_file(river_network_shp).to_crs(output_crs) # import OSM river network data and reproject
river_network=river_network.loc[river_network['waterway'].isin(['canal','river'])] # select river network by attribute

# hand layer
hand=xr.open_dataset(hand_raster,engine="rasterio").squeeze()

# google bulding layer
google_buildings=gpd.read_file(google_building_shp).to_crs(output_crs)
google_buildings=google_buildings.loc[google_buildings['confidence']>=0.6] # filter out low confidence polygons

### loop through tiles for reclassification and export as geotiffs

In [ ]:
# loop through all tiles for reclassification across the country
for i in range(len(tile_bboxes)):
    
    # get tile bbox
    x_min,y_min,x_max,y_max=tile_bboxes.iloc[i]
    print('Processing tile ',i,'with bbox of ',x_min,y_min,x_max,y_max)
    
    # clip land cover map to tile boundary
    landcover2021_tile=landcover2021.rio.clip_box(minx=x_min,miny=y_min,maxx=x_max,maxy=y_max)
    ds_geobox=landcover2021_tile.geobox # get bbox
    np_landcover2021=landcover2021_tile.squeeze().to_numpy() # data array to numpy array
    np_landcover2021_post=np_landcover2021.copy() # initialise reclassified numpy array
    
    # mode filtering for a smoother classification map
    np_landcover2021_post=modal(np_landcover2021,selem=disk(2),mask=np_landcover2021!=0)
    
    # load  and clip hand layer
    hand=xr_reproject(hand, ds_geobox, resampling="average")
    np_hand=hand.to_array().squeeze().to_numpy()
    
    # Make sure water is (only occuring at bottom of watersheds) or fallen within OSM river networks
    river_network_mask=xr_rasterize(gdf=river_network,
                                      da=landcover2021_tile.squeeze(),
                                      transform=ds_geobox.transform,
                                      crs=output_crs)
    np_river_network_mask=river_network_mask.to_numpy()
    np_landcover2021_post[((np_landcover2021==dict_map['Water Body'])&(np_hand<=45))|(np_river_network_mask==1)]=dict_map['Water Body']
    
    # assign pixels overlapping google building polygons as built-up
    google_buildings_mask=xr_rasterize(gdf=google_buildings,
                                      da=landcover2021_tile.squeeze(),
                                      transform=ds_geobox.transform,
                                      crs=output_crs)
    np_google_buildings_mask=google_buildings_mask.to_numpy()
    np_landcover2021_post[np_google_buildings_mask==1]=dict_map['Urban Settlement']
    
    # reclassify wetlands around (within 50m of) built-up areas as annual croplands
    urban_buffered=binary_dilation(np_landcover2021==dict_map['Urban Settlement'],selem=disk(5))
    np_landcover2021_post[(urban_buffered==1)&(np_landcover2021==dict_map['Wetland'])]=dict_map['Annual Cropland']
    
    # assign pixesl overlapping OSM road network as built-up class
    road_network_mask=xr_rasterize(gdf=road_network,
                                  da=landcover2021_tile.squeeze(),
                                  transform=ds_geobox.transform,
                                  crs=output_crs)
    np_road_network_mask=road_network_mask.to_numpy()
    np_landcover2021_post[np_road_network_mask==1]=dict_map['Urban Settlement']
    
    # convert reclassified numpy array result back to DataArray
    landcover2021_tile_post=xr.DataArray(data=np_landcover2021_post,dims=['y','x'],coords={'y':landcover2021_tile.y.to_numpy(), 'x':landcover2021_tile.x.to_numpy()})
    landcover2021_tile_post.rio.write_crs(output_crs, inplace=True)
    
    # export as geotiff
    write_cog(landcover2021_tile_post, 'Results/Land_cover_prediction_Rwanda_2021_using_2015_sheme2_stratified_samples_balanced_post-processed_smaller_tile_'+str(i)+'.tif', overwrite=True)
#     write_cog(landcover2021_tile_post, 'Results/Land_cover_prediction_Rwanda_2021_using_2015_sheme2_stratified_samples_balanced_post-processed_AOI_'+str(i)+'.tif', overwrite=True)

### mosaic all post-processed tiles

In [3]:
# merge into a mosaic once all tiles are processed
! gdal_merge.py -o Results/Land_cover_prediction_Rwanda_2021_using_2015_sheme2_stratified_samples_balanced_post-processed_smaller_mosaic.tif -co COMPRESS=Deflate -ot Byte Results/Land_cover_prediction_Rwanda_2021_using_2015_sheme2_stratified_samples_balanced_post-processed_smaller_tile_*.tif

0...10...20...30...40...50...60...70...80...90...100 - done.
